# Image Classification using Convolutional Neural Networks 

In light of the exponential increase and computer power coupled with generation and storage of big data, the field of AI has taken the spot light in both academic and industry. In particular we have advacements in deep learning(explained further in the upcoming section) and meaningful high impact applications across all sorts of field. Deep Learning models are being used to model extremely complicated networks, make accurate regression and classification predictions, and even generate art. In this tutorial, we cover a particular type of Deep Learning architecutre called Convlutional Neural Network to build an animal classification model. With successful completion of this tutorial, it will be easy to imagine the array of subjects this image classifiction could be used at. For instance, identifing cancerous cells, self-driving car technology, security cameras, iris recognition improvement and so on.

That said within the grander context of AI, Deep Learning can be thouht as a subset of Machine Learning which itself is a subset of AI. 

<img src="../img/ai_ml_dl.png"/>



In this notebook tutorial, we will build a model to classify images into one of five classes: mucca (cow), pecora (sheep), elefante (elephant), farfalla (butterfly) and scoiattolo (squirrel)

The tutorial is sectioned as follows: 
1. Intro to Deep Learning
* Convolutional Neural Networks
2. Data Wrangling
* Handling way too messy folder structures
* Data collection
* Data cleaning
3. Data Exploration 
* TF dataset creation 
* Data visualization
* Dealing with imbalanced dataset 
* Data augementation 
4. Modeling
* TensorBoard for visualization
* Dealing with overfitting
5. Additional Resources</br></br>
6. References

### 1. Intro to Deep Learning
Artifical Neutral Networks(ANN) at the core of deep learning. ANNs are inspired by the netwoks of biological neurons found in our brains. We'll not go into detail about the histotry and nature of ANNs but it suffices to say they are fundamentally different from traditional Machine Learning models. 

One of the simplest ANN architectures is called the Perceptron. Below if a figure of a perceptron. 

<img src="../img/perceptron.png" style="width:300px; height: 200px"/>


The inputs and output are numbers (instead of binary on/off values), and each input connection is associated with a weight. The unit computes a weighted sum of its inputs (z = w1 x1 + w2 x2 + ⋯ + wn xn = x⊺ w), then applies a step function to that sum and outputs the result: hw(x) = step(z), where z = x⊺ w. All of this is to say for given inpute the neuron process it and returns an output.



We should not that neurons are the units of ANNs. An amalgam of these neurons creates a layer. As such we have input layers and output layers. Each ANN model contains hidden layers as well. The more the number of the layers, the deeper the neural network, hence the phrase "Deep Learning". 


#### 1.1 Convolutional Neural Networks

In this project we'll employ a type of ANNs called Convolutional Neural Networks(CNNs). CNNs are widely used in image classification tasks. 

The purpose of this notebook is to train a Deep Learning model that is trained on thousands of labeled data and is able to predict an image into one of said five classes. The project was set up as a challenge at [DPHI](https://dphi.tech/challenges/data-sprint-51-predict-the-image-of-the-species/167/overview/about)

### 2. Data Collection

#### 2.1 Handling way too messy folder structures

Before we start writing our code, it's a good practice to set up our directory structure. We will use the widely used [cookie clutter](https://drivendata.github.io/cookiecutter-data-science/) approach. We will modify the recommended structure for our purposes. We won't have any interim files nor will we save our processed file or use external data. Instead, we will just have a directory named raw which will contain our downloaded file. 

The dataset is stored and shared on a Google Drive folder as a zip file. We will automate the task of downloading, moving to a preffered directory, unzipping and renaming tasks using Python. We will need two libraries for these tasks: gdown and os. 

Uncomment and run the following cell to install the required libraries. 

In [ ]:
# !pip install pandas
# !pip install numpy
# !pip install seaborn
# !pip install gdown
# !pip install tensorflow
# !pip install keras


If you are a fan of `Zen` mode on VSCode you might want to uncomment, run and see if you like the workspace.


In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

#### 2.2 Getting the data

The given dataset is located at https://drive.google.com/file/d/176E-pLhoxTgWsJ3MeoJQV_GXczIA6g8D/view?usp=sharing
We can either download the zipped file from our brower or use the library gdown and download the file using python. 
We choose the latter option. The gdown library requires that we remove extraneous information in the url including view and usp parameters. We only need the id. 

In [ ]:
import gdown

# Download the file to the current working directory
url = 'https://drive.google.com/uc?id=176E-pLhoxTgWsJ3MeoJQV_GXczIA6g8D'
output = '../data/raw/animal_ds.tgz'
gdown.download(url, output, quiet=False)


In [ ]:
# Unzip the file and move it to the correct directory.
!unzip '../data/raw/animal_ds.tgz' -d ../data/raw 



In [ ]:
# Remove the zipped file
!rm ../data/raw/animal_ds.tgz


Now that our data sub-directory is created and contains the data, we will save its path for future use. 


In [ ]:
import pathlib

data_dir = "../data/raw/animal_dataset_intermediate/train"
data_dir = pathlib.Path(data_dir)


Each one of the sub-folders under the train sub-sub-directory contain the '_train' substring in their name. We will use the os library to locate and rename the folders. We do so because tensorflow automatically extracts the class names from the folder names. This will explained after a few cells.


In [ ]:
import os

def rename_files(): 
    for filename in os.listdir(data_dir):
        full_filepath = os.path.join(data_dir, filename)
        os.rename(full_filepath, full_filepath[:-6])
rename_files()


#### 2.3 Data cleaning

The only other action we will perform on our raw file is deletion. We only want to include images with certain file formats(e.g. *.jpg and *jpeg) we can easily work with .

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
num_skipped = 0
for folder_name in ("elefante", "farfalla", "mucca", "pecora", "scoiattolo"):
    folder_path = os.path.join(data_dir, folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)



The cell above deleted files if they are not of the type we can work with. It also printed the number of the files deleted. 

Now we can view the number of images available in the train sub-directory. 

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')) + list(data_dir.glob('*/*.jpeg')))
print("Imported image_count: ", image_count)



Before reading the files in a manner suitable for our modeling, we will view the first file just to see if it's actually an image. 


In [ ]:
import PIL
import PIL.Image


In [ ]:
elephant = list(data_dir.glob('elefante/*'))
PIL.Image.open(str(elephant[0]))


### 3. Data Exploration

We will be using Tensorflow a deep learning library. We will also be using Keras, an API that let's as access Tensorflow. 

(TensorFlow)[https://www.tensorflow.org/] is a free and open-source software library for machine learning and artificial intelligence. It can be used across a range of tasks but has a particular focus on training and inference of deep neural networks.

[Keras](https://keras.io/) is a high-level Deep Learning API that makes it easy to build, train, evaluate, and execute all sorts of neural networks. It was developed by (François Chollet)[https://fchollet.com/] as an open source project in March 2015. It quickly gained popularity, owing to its ease of use, flexibility, and beautiful design. 

FYI: The most popular Deep Learning library, after Keras and TensorFlow, is Facebook’s PyTorch library.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



In any Machine Learning project, we always have two distinct sets of our data. *Training set* and *Testing set*. Sometimes we have a third set called *Validation set* 

* Training set - contains the majority of the data and is used to train our model. The percentage of data allocated to this set can vary. Depending on the size of our data, we might allocate 70% - 90% of our dataset. The more data we have the more percentage we can allocate to our training set. 
* Validation set - much like training set, it contains data with both features and lables. It is used to measure the accuracy of the model i.e. how good does the model predict unseen data? 
* Testing set - This is feature only data for which our model will predict labels.

#### 3.1 TF dataset creation

The`image_dataset_from_directory`(main_directory, labels='inferred') from Keras will return a tf.data.Dataset that yields batches of images from the subdirectories elefant, ..., scoiattolo. together with labels 0 to 4 (0 corresponding to elefante and 4 corresponding to scoiattolo).

It supports the following image formats: jpeg, png, bmp, gif. Hence the need to remove unsupported format which we have already done.

The`image_dataset_from_directory` accepts dataset folder divides it up to training and validation set. More importantly, the images are assigned into batches. In our case, we set the batch size to 32. If you have a low performing machine you can set it to 16. The image height and image width are set following the instruction from the compeition. The origianl shpae of the images is 256 by 256 so we will stick with that. 

<img src="../img/CNN_input_shape.png" style="width:400px; height: 300px"/>


In [ ]:
batch_size = 32
img_height = 256
img_width = 256


train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2, # We train our model using 80% of the train_ds and test on the remaining 20%.
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)



The `image_dataset_from_directory` method created a TF object from our files. It also inferred the numbr of classes in both the training and validation sets, correctly so. 


In [ ]:
class_names = ['elefante', 'farfalla', 'mucca', 'pecora', 'scoiattolo']


#### 3.2 Data visualization

We can read and generate images from our TF object. Data visualization is not a one-off task. We are going to generate different figures as needed.

The `take()` methods accepts an float number that refers to the batch. Batching starts at 1 in TF. So, .take(1) refers to the first batch containing 32 images. 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
    plt.show()



One of the trickiest challenges of working with images is resizing shapes. 
Note: We are treating the images as TF objects contained in batched datasets 

Despite the convenience, batched datasets can be a bit tricky especially when we want to apply changes on individual images. Here will try to resample our data because we have an imbalanced dataset. To do so, we first iterate through each batch and save each image as a numpy array and lables as another numpy array.

In [ ]:
# Get images(x) and lables(y) of a given batchdatasets
def get_image_label(ds): 
    x_train_ = []
    y_train_ = []
    for element in ds.as_numpy_iterator(): 
        x_train_.append(element[0])
        y_train_.append(element[1])
    x_train = np.concatenate(x_train_)
    y_train = np.concatenate(y_train_)
    
    return (x_train, y_train)

x_train, y_train = get_image_label(train_ds)
x_val, y_val = get_image_label(val_ds)

print(type(x_train), type(y_train))
print(x_train.shape, y_train.shape, x_train.ndim)
print(x_val.shape, y_val.shape, x_val.ndim)


It is a good practice to test objects and their attributes especially when casting them. In fact, in the previous cell we not only changes the dtype of the object we are working with but also which class it belongs to i.e. from TF batched dataset to numpy arrays. We expect that x_train which is the array for the feature. It should be four-dimensional as it the original object contained a four-dimensional object (batch_size, img_height, img_width, xxx)

In [ ]:
assert isinstance(x_train, (np.ndarray, np.generic))
assert isinstance(y_train, (np.ndarray, np.generic))
assert isinstance(x_val, (np.ndarray, np.generic))
assert isinstance(y_val, (np.ndarray, np.generic))

assert x_train.ndim, x_val == 4
assert y_train.ndim, y_train.ndim == 1


#### 3.3 Dealing with imbalanced dataset

We can check whether our data is imbalanced by looking the number of images each class contains. To do so, we will use seaborn library to produce a data visualization. Our classes are categorical valus and their counts are continous, so we will use a bar graph. 

In [ ]:
#TODO: Data viz to show data imbalance - bar graph

plt.figure(figsize=(8, 3))
splot = sns.countplot(x = y_train)
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.0f'), (p.get_x() + p.get_width() / 2., p.get_height()), 
    ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

splot.set_xticklabels(class_names)
splot.set_xlabel("Classes")
splot.set_ylabel("Count")
plt.show()


There are multiple ways to deal with an imbalanced dataset: 

1. We can assign a larger penalty to wrong predictions on the minority class. 
2. We can upsample the minority class, 
3. We can downsample the majority class
4. We can generate synthetic training examples.

The most widely used algorithm for synthetic training data generation is Synthetic Minority Oversampling Technique (SMOTE). 

Of all the last point is probably the most complex to understand. Nonetheless, it is already implemented in imbalanced-learn a Python library that is entirely focused on imbalanced datasets. We are then at the luxury of using this technique.

In [ ]:
from sklearn.utils import resample 

print("-" * 70)
print('Input shape before resampling: ' ,x_train.shape, y_train.shape)

#..reshape (flatten) x_train for SMOTE resampling
nsamples, k, nx, ny = x_train.shape
x_train = x_train.reshape((nsamples,k*nx*ny))
x_train.shape

from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='all')
x_train, y_train = smote.fit_resample(x_train, y_train)

print("-" * 70)
print('Input shape after sampling: ' ,x_train.shape, y_train.shape)
print('Class distribution after over-sampling: ')
for i in range(len(class_names)):
    print(f'Number of class {class_names[i]} examples before:{x_train[y_train == i].shape[0]}')
    

Now that we used SMOTE to resample our data, we can check if our training dataset contains equal number of images across the five classes. 

In [ ]:
plt.figure(figsize=(8, 3))
splot = sns.countplot(x = y_train)
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.0f'), (p.get_x() + p.get_width() / 2., p.get_height()), 
    ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

splot.set_xticklabels(class_names)
splot.set_xlabel("Classes")
splot.set_ylabel("Count")
plt.show()


Having completed our resampling, we can now restore the images to their original four-dimensional shape.

In [ ]:
# return to original 4D shape
x_train = x_train.reshape(7325, k, nx, ny)


##### Performance enhancement

Training deep learning models can be a computationally expensive task. There are several ways to better our modeling performance, one is prefetching. Prefetching generally refers to the use of a background thread and an internal buffer to prefetch elements from the input dataset ahead of the time they are requested. The details are not particularly of interest to us at the moment. Besides, we can let scikit-learn optimize the parameters.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Another difference between traditional machine learning and deep learning is that deep learning requires huge amounts of data. There is no particular number we're looking at to decide whether we should use one or the other. Generally speaking so he wants quite a bit of data in either case and more so for deep learning.

That being said around dataset contains around 8000 images and there is not necessarily enough to train a deep learning model or at least a fairly accurate one. In addition, in the real world scenario, as opposed to certain conditions our target application may exist in a variety of conditions, such as different orientation, location, scale, brightness etc. To account for these situations, we can use data augementation, which is a technique of training our neural network with additional synthetically modified data.

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal", input_shape=(img_height, img_width,3)),
    layers.RandomRotation(0.1),layers.RandomZoom(0.1),
    layers.RandomContrast((0.1, 0.9)),
    ])

assert(x_train.ndim == 4) # Check if augementation affected shape


### 4. Modeling
Our dataset has been cleaned, resized, balanced, and augemented. Now, we can safely move to the modeling phase. Here we define our Neural Network structure. By defining we mean choosing the architeture along with number of neurons and layers, and choosing activation function. We then compile our model which includes choosing the optimizer, loss function and metrics. We have defined neurons and layers but what are optimizers and loss functions?

As opposed to traditional ML, in Deep Learning we let the neurons find out the weight and biases of each neuron. Naturally, then we need a methods to decide which of these weight and biase parameters result in a more accurate model. There are two components that let's us do exactly that. Loss functions computes the distance between the current output of the algorithm and the expected output, then return a single value that we can compare with other weight and biase choices. Optimizers are mathematical functions which are dependent on model's learnable parameters i.e. weights & biases.

Delving into all the loss functions and optimizers is beyond the scope of this tutorial. We just need to note that we'll use Adam optimiser and SparseCategoricalCrossentropy because we have multi-class labels.

#### Tensorboard for visualization

TensorBoard is a great interactive visualization tool that can show the learning curves during training, compare learning curves between multiple runs, visualize the computation graph, analyze training statistics, view images generated by the model, visualize complex multidimensional data projected down to 3D and more! 

TensorBoard might launch in a different port so you don't have to worry about your running jupyter notebooks. 
Below are figures of screenshots taken while training.

<img src="../img/screenshot_tensorboard_visualization.png" style="width:1100px; height: 600px"/>


In [ ]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir() # e.g., './my_logs/run_2019_06_07-15_15_22'

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006


You might notice here we have pooling features. A definition from the popular *machinelearningmastery* blog can help with that.

<blockquote>A problem with the output feature maps is that they are sensitive to the location of the features in the input. One approach to address this sensitivity is to down sample the feature maps. This has the effect of making the resulting down sampled feature maps more robust to changes in the position of the feature in the image, referred to by the technical phrase “local translation invariance.”
</blockquote> 

In [ ]:
num_classes = 5

model = tf.keras.Sequential([
  data_augmentation,
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  layers.Dropout(0.2), # Dropout https://www.tensorflow.org/tutorials/images/classification#dropout
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])


In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy']
  )


#### Dealing with overfitting

Two callbacks options to implement early Stopping to avoid overfitting. 

1. keras.callbacks.ModelCheckpoint() saves the model when its performance on the validation set is the best so far

2. keras.callbacks.EarlyStopping() interrupts training when it measures no progress on the validation set for a number of epochs (defined by the patience argument), and it will optionally roll back to the best model.

It's possible to  combine both callbacks to save checkpoints of your model (in case the computer crashes) and interrupt training early when there is no more progress (to avoid wasting time and resources)

Despite our attempt to enhance the model traning performance, it might take us a lot of time to complete the training. If GPUs or TPUs are available then changing the runtime is highly recommended. If not we can load a saved model for our convenience.

In [ ]:
epochs = 50
checkpoint_cb = keras.callbacks.ModelCheckpoint("saved_model/keras_model.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history =  model.fit(
  x_train, y_train,
  validation_data=(x_val, y_val),
  epochs=epochs,
  callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb]
)


checkpoint_cb saves the best model. Just in case, though, we'll also save the final model

Save your model in the appropriate directory

In [ ]:
model.save('../saved_model/best_model') 


Evalute your model on the validation set. If you load the saved model


In [ ]:
model.evaluate(x_val, y_val, verbose=2) # ~78% accuracy


Uncomment and run the following cell to load a saved model with an approximate accuracy of 78%.

In [ ]:
# new_model = tf.keras.models.load_model('../my_keras_model.h5')
# new_model.evaluate(x_val, y_val, verbose=2) # ~78% accuracy


Congrats! We have a working model. Typically this would mark the end of our journey, but our aim is to predict a test set (one with no labels) and submit it to the competition. We will do just that.

In [ ]:
import pandas as pd

pd.options.display.max_colwidth = 999


In [ ]:
test_data_dir = "../data/raw/animal_dataset_intermediate/test"
test_data_dir = pathlib.Path(test_data_dir)
print(test_data_dir)

image_count = len(list(test_data_dir.glob('*.jpg')) + list(test_data_dir.glob('*.jpeg')))
print("Imported image_count: ", image_count)

picture = list(test_data_dir.glob('*'))
PIL.Image.open(str(picture[0]))


The challenge orgnizers want us to submit our predictions as a csv file wth headers filename and target. The order for the filename column is already given. We will iterate through the given `Testing_set_animals.csv` file predict the class and put in our predictions in the `submissions.csv` file.

In [ ]:
df_submission_filename = pd.read_csv("../data/raw/animal_dataset_intermediate/Testing_set_animals.csv")

classes = []

df_len = df_submission_filename.shape[0]
for i in range(df_len): 
    path = os.path.join(test_data_dir, df_submission_filename.loc[i][0])
    img = tf.keras.utils.load_img(
        path, grayscale=False, color_mode='rgb', target_size=(img_height, img_width),
        interpolation='nearest'
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = img_array.reshape(1, k, nx, ny)
    predict_img = model.predict(img_array) 
    classes_img = np.argmax(predict_img,axis=1)
    classes.append(classes_img[0])


In [ ]:
df_submission_filename['target'] = [class_names[num] for num in list(classes)]
df_submission_filename.head()


In [ ]:
df_submission_filename.to_csv("../submission/submission.csv", index=False)


### Additional Resources

Apart from the hyperlinks presented above and the refernces listed below, here are additonal resources you can check:
- [Create your Own Image Classification Model using Python and Keras](https://www.analyticsvidhya.com/blog/2020/10/create-image-classification-model-python-keras/#h2_7)
- [Building powerful image classification models using very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
- [Build your First Multi-Label Image Classification Model in Python](https://www.analyticsvidhya.com/blog/2019/04/build-first-multi-label-image-classification-model-python/)
- [Top 4 Pre-Trained Models for Image Classification with Python Code](https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/)
- [Complete Guide To Bidirectional LSTM (With Python Codes)](https://analyticsindiamag.com/complete-guide-to-bidirectional-lstm-with-python-codes/)
- [Advanced CNN Architectures](https://livebook.manning.com/book/grokking-deep-learning-for-computer-vision/chapter-5/v-3/9)

### References

Brownlee, J. (2019, July 5). Best practices for preparing and augmenting image data for cnns. Machine Learning Mastery. Retrieved October 28, 2021, from https://machinelearningmastery.com/best-practices-for-preparing-and-augmenting-image-data-for-convolutional-neural-networks/. 

Brownlee, J. (2020, August 14). What is the difference between test and validation datasets? Machine Learning Mastery. Retrieved October 28, 2021, from https://machinelearningmastery.com/difference-test-validation-datasets/. 

Brownlee, J. (2021, March 16). Smote for imbalanced classification with python. Machine Learning Mastery. Retrieved October 28, 2021, from https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/. 

Géron Aurélien. (2019). Hands-on machine learning with scikit-learn and tensorflow concepts, tools, and techniques to build Intelligent Systems. O'Reilly. 

Jupyter Notebook markdown tutorial. DataCamp Community. (n.d.). Retrieved October 28, 2021, from https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook. 

Keras documentation: Image Classification From Scratch. Keras. (n.d.). Retrieved October 28, 2021, from https://keras.io/examples/vision/image_classification_from_scratch/. 

Load and preprocess images &nbsp;: &nbsp; Tensorflow Core. TensorFlow. (n.d.). Retrieved October 28, 2021, from https://www.tensorflow.org/tutorials/load_data/images. 

Raschka, S., &amp; Mirjalili, V. (2019). Python machine learning: Machine learning and deep learning with python, scikit-learn, and tensorflow 2. Packt Publishing. 

Verma, S. (2021, October 5). Understanding input and output shapes in convolution neural network: Keras. Medium. Retrieved October 28, 2021, from https://towardsdatascience.com/understanding-input-and-output-shapes-in-convolution-network-keras-f143923d56ca. 